# Wrangle (Acquire and Prepare)

This notebook contains all steps and decisions in the data acquisition and data preparation phases of the pipeline.

---

## The Required Modules

Below are all the modules needed to run the code cells in this notebook.

In [26]:
import pandas as pd

from univariate_analysis import *
from prepare import *

---

## Data Acquisition

Before we can begin doing anything the data must be acquired. The data can be acquired from either the NASA Exoplanet Archive or from Kaggle. We'll walk through both methods of acquiring the data.

### Download With Kaggle API

The next cell will download the data using the kaggle API. The downloaded file will be a .zip file. We will need to unzip the file and then we can read the .csv file into a pandas dataframe.

This of course assumes that the Kaggle API is installed on the local machine. For instructions on downloading the data without the Kaggle API skip ahead to the "Download Without Kaggle API" section. Otherwise, if the Kaggle API is not installed, follow the instructions outlined in the Kaggle API repository README [here](https://github.com/Kaggle/kaggle-api) or follow the steps below:
- On the command line run 

```bash
    pip install kaggle
```

- Login to Kaggle, go to Your Profile -> Account -> API click "Create New API Token"
- Move the downloaded kaggle.json file to ~/.kaggle/kaggle.json, 

```bash
    mv ~/Downloads/kaggle.json ~/.kaggle/kaggle.json
```

- On the command line run 

```bash
    chmod 600 ~/.kaggle/kaggle.json
```

Now we can run the commands below to download the data.

In [9]:
# Download the source data, unzip the downloaded file, and remove the zip file.

!kaggle datasets download nasa/kepler-exoplanet-search-results -f cumulative.csv
!unzip cumulative.csv.zip
!rm cumulative.csv.zip

 86%|████████████████████████████████▊     | 1.00M/1.16M [00:00<00:00, 6.11MB/s]
100%|██████████████████████████████████████| 1.16M/1.16M [00:00<00:00, 7.02MB/s]
Archive:  cumulative.csv.zip
  inflating: cumulative.csv          


In [10]:
# Now let's read the data into a dataframe.

kepler = pd.read_csv('cumulative.csv')
kepler.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rowid              9564 non-null   int64  
 1   kepid              9564 non-null   int64  
 2   kepoi_name         9564 non-null   object 
 3   kepler_name        2294 non-null   object 
 4   koi_disposition    9564 non-null   object 
 5   koi_pdisposition   9564 non-null   object 
 6   koi_score          8054 non-null   float64
 7   koi_fpflag_nt      9564 non-null   int64  
 8   koi_fpflag_ss      9564 non-null   int64  
 9   koi_fpflag_co      9564 non-null   int64  
 10  koi_fpflag_ec      9564 non-null   int64  
 11  koi_period         9564 non-null   float64
 12  koi_period_err1    9110 non-null   float64
 13  koi_period_err2    9110 non-null   float64
 14  koi_time0bk        9564 non-null   float64
 15  koi_time0bk_err1   9110 non-null   float64
 16  koi_time0bk_err2   9110 

Now we have our exoplanet data and we can already see that some cleaning is needed. We'll discuss this more in the preparation section.

### Download Without Kaggle API

For reproducibility we must consider that some people may not have the Kaggle API installed and may not want to install and set it up. So let's go through the steps of downloading the data without the Kaggle API.

<i>This is the process for using the NASA API to download the data, but at this time will not be made into a programmatic solution.</i>

In [11]:
# import requests

# url = 'https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?'
# endpoint = '?table=cumulative'
# query = '&select=defaults'
# response_format = '&format=json'

# data = requests.get(url + endpoint + query + response_format).json()

### Automate The Download Procedure

Now let's automate the acquisition procedure for ease of use.

In [12]:
# We'll need a way to run the kaggle api commands with python. We can use the os module for this.

import os

In [13]:
# # This cell can be executed to remove the existing cumulative.csv file.
# !rm cumulative.csv

In [14]:
# This code will download the kepler data, unzip the file, remove the zip file, and read the data into a dataframe

# os.system('kaggle datasets download nasa/kepler-exoplanet-search-results -f cumulative.csv')
# os.system('unzip cumulative.csv.zip')
# os.system('rm cumulative.csv.zip')
# kepler = pd.read_csv('cumulative.csv')
# kepler.info()

That works, but what if the Kaggle API is not installed. How will we ensure that the proper error message is raised.

In [15]:
# os.system will return 0 if the command was successful. We will need to check if the return value was 0,
# and otherwise raise an exception.

output = os.system('invalid command')
if output != 0:
    raise SystemError('''
        An error occurred when running "kaggle datasets download".
        You must either follow the instructions for installing the Kaggle API
        here https://github.com/Kaggle/kaggle-api or manually download the 
        data from here https://exoplanetarchive.ipac.caltech.edu/docs/data.html
    ''')

sh: invalid: command not found


SystemError: 
        An error occurred when running "kaggle datasets download".
        You must either follow the instructions for installing the Kaggle API
        here https://github.com/Kaggle/kaggle-api or manually download the 
        data from here https://exoplanetarchive.ipac.caltech.edu/docs/data.html
    

In [16]:
# and if we use a valid command ...

output = os.system('echo hello')
if output != 0:
    raise SystemError('''
        An error occurred when running "kaggle datasets download".
        You must either follow the instructions for installing the Kaggle API
        here https://github.com/Kaggle/kaggle-api or manually download the 
        data from here https://exoplanetarchive.ipac.caltech.edu/docs/data.html
    ''')

hello


Now let's use the Acquire class to wrap the acquisition process into it's own object for ease of use.

In [17]:
from acquire import Acquire

In [18]:
# # This cell can be executed to remove the existing cumulative.csv file.
!rm cumulative.csv

In [22]:
# This class will allow us to easily acquire the data from wherever we need it.

class AcquireKeplerData(Acquire):
    # The Acquire class handles most of the work for us. We only need to override the
    # read_from_source method.
    def read_from_source(self):
        shell_output = os.system('kaggle datasets download nasa/kepler-exoplanet-search-results -f cumulative.csv')
        if shell_output != 0:
            raise SystemError('''
                An error occurred when running "kaggle datasets download".
                You must either follow the instructions for installing the Kaggle API
                here https://github.com/Kaggle/kaggle-api or manually download the 
                data from here https://exoplanetarchive.ipac.caltech.edu/docs/data.html
            ''')
        
        os.system('unzip cumulative.csv.zip')
        os.system('rm cumulative.csv.zip')
        df = pd.read_csv('cumulative.csv')
        
        return df
    
    def create_cache_file(self, df, cache_data, verbose):
        if cache_data:
            if verbose: print('Cacheing data.')
            os.system(f'mv cumulative.csv {self.file_name}')
        else:
            if verbose: print('Data not cached.')
            os.system('rm cumulative.csv')

In [45]:
# Let's test it.

acquire = AcquireKeplerData('kepler.csv')
kepler = acquire.get_data(verbose = True)
kepler.info()

Reading from .csv file.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rowid              9564 non-null   int64  
 1   kepid              9564 non-null   int64  
 2   kepoi_name         9564 non-null   object 
 3   kepler_name        2294 non-null   object 
 4   koi_disposition    9564 non-null   object 
 5   koi_pdisposition   9564 non-null   object 
 6   koi_score          8054 non-null   float64
 7   koi_fpflag_nt      9564 non-null   int64  
 8   koi_fpflag_ss      9564 non-null   int64  
 9   koi_fpflag_co      9564 non-null   int64  
 10  koi_fpflag_ec      9564 non-null   int64  
 11  koi_period         9564 non-null   float64
 12  koi_period_err1    9110 non-null   float64
 13  koi_period_err2    9110 non-null   float64
 14  koi_time0bk        9564 non-null   float64
 15  koi_time0bk_err1   9110 non-null   float64
 16  

Now we can throw this code into it's own file to use in the final report or anywhere else it might be needed.

---

## Data Preparation

Now that we've acquired the data we need to prepare it before we can begin analysis. In order to prepare the data we'll want to handle any missing values either by removing them or imputing them. We can also drop any columns that are undeniably useless for achieving our goals and we'll probably want to rename the columns for readability.

### Handling Missing Values

Let's start by handling the missing values. We'll look at the number of missing values in each column and determine a best course of action from there.

In [25]:
# Let's see the number of missing values in each column.

summarize_column_nulls(kepler)

,rows_missing,percent_missing
rowid,0,0.000000
kepid,0,0.000000
kepoi_name,0,0.000000
kepler_name,7270,0.760142
koi_disposition,0,0.000000
koi_pdisposition,0,0.000000
koi_score,1510,0.157884
koi_fpflag_nt,0,0.000000
koi_fpflag_ss,0,0.000000
koi_fpflag_co,0,0.000000


koi_teq_err1 and koi_teq_err2 are both completely empty so these can be dropped. The kepler_name column is also mostly empty. The data dictionary describes this feature as a more memorable name given to confirmed exoplanets. This column won't be helpful to our goal either way so it can be dropped. The koi_score column is missing 15% of its values. The data dictionary describes this feature as the confidence of the disposition. This feature may possibly be useful so we'll drop all rows missing this column.

**First Pass**
What we can do is simply drop nulls by percentage, that is remove all columns missing a certain percentage of values, we'll go with 75% to make sure the kepler_name column gets dropped and we'll drop all rows that remain with missing values. 

In [28]:
kepler.shape

(9564, 50)

In [36]:
# drop columns with 75% or more missing values, drop all rows missing values.
drop_missing_values(kepler, 0.75, 1).shape

(7803, 47)

This method still leaves us with most of the columns and a reasonable number of observations.

In [34]:
print(f'Percentage of columns remaining: {47 / 50:.0%}')

Percentage of columns remaining: 94%


In [35]:
print(f'Percentage of rows remaining: {7803 / 9564:.0%}')

Percentage of rows remaining: 82%


In [37]:
# Now let's assign the resulting dataframe to our kepler variable.
kepler = drop_missing_values(kepler, 0.75, 1)
summarize_column_nulls(kepler)

,rows_missing,percent_missing
rowid,0,0.0
kepid,0,0.0
kepoi_name,0,0.0
koi_disposition,0,0.0
koi_pdisposition,0,0.0
koi_score,0,0.0
koi_fpflag_nt,0,0.0
koi_fpflag_ss,0,0.0
koi_fpflag_co,0,0.0
koi_fpflag_ec,0,0.0


In [38]:
kepler.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7803 entries, 0 to 9563
Data columns (total 47 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rowid              7803 non-null   int64  
 1   kepid              7803 non-null   int64  
 2   kepoi_name         7803 non-null   object 
 3   koi_disposition    7803 non-null   object 
 4   koi_pdisposition   7803 non-null   object 
 5   koi_score          7803 non-null   float64
 6   koi_fpflag_nt      7803 non-null   int64  
 7   koi_fpflag_ss      7803 non-null   int64  
 8   koi_fpflag_co      7803 non-null   int64  
 9   koi_fpflag_ec      7803 non-null   int64  
 10  koi_period         7803 non-null   float64
 11  koi_period_err1    7803 non-null   float64
 12  koi_period_err2    7803 non-null   float64
 13  koi_time0bk        7803 non-null   float64
 14  koi_time0bk_err1   7803 non-null   float64
 15  koi_time0bk_err2   7803 non-null   float64
 16  koi_impact         7803 

That took care of all the missing values.

### Dropping Additional Columns

There are some id columns that will likely not be very useful to us. Let's first see what rowid and kepid are.

In [39]:
kepler[['rowid', 'kepid']].head()

,rowid,kepid
0,1,10797460
1,2,10797460
2,3,10811496
3,4,10848459
4,5,10854555


Now let's see what kepoi_name is.

In [40]:
kepler.kepoi_name.head()

0    K00752.01
1    K00752.02
2    K00753.01
3    K00754.01
4    K00755.01
Name: kepoi_name, dtype: object

It looks like rowid is not much different than the index so we can drop it. kepid and kepoi_name are unique identifiers for objects of interest. These won't be useful to us either so we can drop all these columns.

In [41]:
kepler = kepler.drop(columns = ['rowid', 'kepid', 'kepoi_name'])
kepler.shape

(7803, 44)

In [42]:
print(f'Percentage of columns remaining: {44 / 50:.0%}')

Percentage of columns remaining: 88%


### Renaming Columns

**First Pass**
There are a lot of columns here and renaming them now might be a tall task. We'll probably be better off exploring our data first and then renaming any features that prove to be useful. So for now we won't rename any columns, except the target variable. We'll rename it to disposition.

In [43]:
kepler = kepler.rename(columns = {'koi_disposition' : 'disposition'})
kepler.columns

Index(['disposition', 'koi_pdisposition', 'koi_score', 'koi_fpflag_nt',
       'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period',
       'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_insol', 'koi_insol_err1',
       'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_tce_delivname', 'koi_steff', 'koi_steff_err1', 'koi_steff_err2',
       'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad',
       'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

### Put it in a Function

Now let's put all these steps into an easy to use function.

In [44]:
def prepare_kepler(df):
    df = drop_missing_values(df, 0.75, 1)
    
    columns_to_drop = [
        'rowid',
        'kepid',
        'kepoi_name'
    ]
    df = df.drop(columns = columns_to_drop)
    
    rename_map = {
        'koi_disposition' : 'disposition'
    }
    df = df.rename(columns = rename_map)
    
    return df

In [47]:
# Let's test it
prepare_kepler(acquire.get_data()).shape

(7803, 44)

It works.